# B-Tree

## Задача B: Высота дерева

```
Реализуйте бинарное дерево поиска для целых чисел. Программа получает на вход последовательность целых чисел и строит из них дерево. Элементы в деревья добавляются в соответствии с результатом поиска их места. Балансировка дерева не производится.

Формат входных данных
На вход программа получает последовательность различных натуральных чисел меньших 10000.

Формат выходных данных
Выведите единственное число – высоту получившегося дерева. 
```

Примеры:
```
Вход:  5 15 18
Выход: 3

Вход:  12 26 23 28 33 8
Выход: 4

Вход:  35 8 6 48 30 17 4 43 42
Выход: 4
```

Будем представлять дерево в виде словарика такого вида:
```
d = {
    root_value: [left_value, right_value],
    ...
}
```

"Словарное" представление возможно, потому что по условию задачи числа, которые предполагается хранить в дереве, *различны*.

(А вообще для работы с деревом можно бы было ещё использовать структуру типа связного списка: набор узлов `Node`, каждый узел хранит значение `value` и ссылки на левого `left` и правого `right` потомков.
Само дерево `Tree` определяется выбором корневого узла `root`.)

In [4]:
s = input()

values = [
    int(v) for v in s.split()
]

tree = {}
root = None

# Будем каждый раз при добавлении нового значения смотреть на высоту,
# на которой будет создаваться узел с этим значением
max_height = 0

for v in values:
    if len(tree) == 0:
        root = v  # Старт -- корень дерева
        tree[root] = [None, None]
        max_height = 1
        
        continue

    current_root = root
    current_height = 1

    is_from_left  = lambda x, r: x <= r
    is_from_right = lambda x, r: x > r
    is_left_nonempty  = lambda r: tree[r][0] is not None
    is_right_nonempty = lambda r: tree[r][1] is not None

    # Ищем свободное место
    while (is_from_left(v, current_root) and is_left_nonempty(current_root)         # Место слева занято
           or is_from_right(v, current_root) and is_right_nonempty(current_root)):  # Место справа занято
        
        # Перешли в нужное поддерево
        if v <= current_root:
            current_root = tree[current_root][0]
        else:
            current_root = tree[current_root][1]
        
        current_height += 1
    
    # До этого нашли свободное место, но теперь надо вспомнить, где оно: слева или справа
    if is_from_left(v, current_root):
        tree[current_root][0] = v   # Сохранили v как потомок root (левый)
    else:
        tree[current_root][1] = v   # ...                          (правый)
    
    tree[v] = [None, None]  # У самого v потомков пока нет
                            # (вот теперь v -- "полноценный узел")

    current_height += 1
    max_height = max(max_height, current_height)

print(max_height)

 35 8 6 48 30 17 4 43 42


4


## Задача A: Дерево — проверка корректности

```
Проверьте, что предложенное дерево является двоичным деревом поиска, т.е. для каждой вершины в левом поддереве все ключи меньше чем в данной вершине, а в правом - больше.

Формат входных данных
На вход программа получает двоичное дерево в следующем формате: В первой строке одно число n - количество вершин (1 <= n <= 1000). В следующих n строках описания вершин, каждое в виде трех числе через пробел: value, left, right, где value - значение в текущей вершине, left и right - номера ее левого и правого потомков. Если потомка нет то соответствующее значение будет -1. Вершины нумеруются с нуля, нулевая вершина - это корень дерева. Все значения в вершинах различны.

Формат выходных данных
YES - если дерево является двоичным деревом поиска, NO - в противном случае. 
```

Примеры:
```
Вход:
3
5 -1 1
15 -1 2
18 -1 -1

Выход:
YES
```

Построим по описанным входным данным "словарикообразное" дерево, как в прошлой задаче.

In [8]:
n = int(input())

values = []     # Значения в узлах дерева
raw_tree = []   # Двумерный список

for _ in range(n):
    line = input()
    numbers = [int(v) for v in line.split()]
    
    values.append(numbers[0])
    raw_tree.append(numbers)

tree = {}

for numbers in raw_tree:
    value, left_index, right_index = numbers  # Распаковка
    
    left_value = values[left_index] if left_index != -1 else None
    right_value = values[right_index] if right_index != -1 else None
    
    tree[value] = [left_value, right_value]

 4
 5 -1 1
 15 -1 2
 18 3 -1
 17 -1 -1


In [9]:
tree

{5: [None, 15], 15: [None, 18], 18: [17, None], 17: [None, None]}

```
      5
       \
        15
          \
           18
          /
         17
```

Как можно получить всех потомков корня справа?

Можно посмотреть *направо*, если есть потомок, отложить его.
Далее, пройтись по отложенным (который либо есть один, либо нет) и отложить *всех* его непосредственных потомков (так как они оба будут потомками справа для исходного корня).
И далее повторять процесс, пока есть отложенные, которые ещё не "посетили": проходим по ним, добавляя в *очередь* на просмотр их потомков, если есть.
Таким образом "лавинообразно" сверху вниз можно пройтись по всему правому (или по аналогии левому) поддереву.

In [13]:
root = 5

descendants = []

get_left  = lambda r: tree[r][0]
get_right = lambda r: tree[r][1]
is_left_nonempty  = lambda r: get_left(r) is not None 
is_right_nonempty = lambda r: get_right(r) is not None

# Проходим потомков справа
if is_right_nonempty(root) is not None:
    descendants.append(get_right(root))

# Пока есть "непросмотренные" потомки
while len(descendants) > 0:
    new_descendants = []
    
    for descendant in descendants:
        # Что-то делаем с descendant (например, проверяем, что он больше root...)
        print(f'Потомок: {descendant} ({descendant > root})')
        
        if is_left_nonempty(descendant):
            new_descendants.append(get_left(descendant))
        
        if is_right_nonempty(descendant):
            new_descendants.append(get_right(descendant))
    
    descendants = new_descendants

Потомок: 15 (True)
Потомок: 18 (True)
Потомок: 17 (True)


На основе приведённого уже можно проверить корректность дерева... (что оно в самом деле двоичное дерево поиска)